In [29]:
#import xml.etree.ElementTree as ET
import lxml.etree as ET
import regex as re
import os
parser = ET.XMLParser(strip_cdata=False,remove_blank_text=False)
ET.register_namespace('DTS',"www.microsoft.com/SqlServer/Dts")
ET.register_namespace('SQLTask',"www.microsoft.com/sqlserver/dts/tasks/sqltask")

#directory='C:\\balaji\\sourcecode\\etl-MPCSEDW\\MPCS_ETL_SSIS\\SSIS\\'
directory='C:\\balaji\\2020\\teradata_migration_analysis\\pkgs\\test\\edit_ssis'

def Check_Delaercomp_Comm(fname):#check  for callidus connections and return file name   
    p = re.compile(r"\L<words>", words=['DealerComp', 'DEALERCOMP', 'CALLIDUS', 'Callidus','Subsidy','SUBSIDY'])
    n = re.compile(r"\L<words>", words=['None'])
    root = ET.parse(fname,parser).getroot()
    callidus=0
    #print(root.tag)
    #print(root.attrib)
    for x in root.iter('{www.microsoft.com/SqlServer/Dts}ConnectionManagers'):
        for y in x.iter('{www.microsoft.com/SqlServer/Dts}ConnectionManager'):
            for z in y.iter('{www.microsoft.com/SqlServer/Dts}ObjectData'):
                for m in z.iter('{www.microsoft.com/SqlServer/Dts}ConnectionManager'):
                     if  p.search(str(m.attrib.get(("{www.microsoft.com/SqlServer/Dts}ConnectionString")))):
                        callidus=1
                        break
    return callidus,fname.split('\\')[-1]                        
def get_mer_connname_ids(fname):    #get mercury connectio name and id 
    q = re.compile(r"\L<words>", words=['Mercury','MERCURY'])
    root = ET.parse(fname,parser).getroot()
    conname_n_id={}
    for x in root.iter('{www.microsoft.com/SqlServer/Dts}ConnectionManagers'):
        for y in x.iter('{www.microsoft.com/SqlServer/Dts}ConnectionManager'):
                if y.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName")) is not None:
                    if  q.search(y.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName"))):
                        conname_n_id[y.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName"))]=y.attrib.get(("{www.microsoft.com/SqlServer/Dts}DTSID"))
                        
                     
    return conname_n_id 

def get_Callidus_connname_ids(fname):    #get Callidus Conn name and id 
    q = re.compile(r"\L<words>", words=['Callidus','CALLIDUS'])
    root = ET.parse(fname,parser).getroot()
    call_conname_n_id={}
    for x in root.iter('{www.microsoft.com/SqlServer/Dts}ConnectionManagers'):
        for y in x.iter('{www.microsoft.com/SqlServer/Dts}ConnectionManager'):
                if y.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName")) is not None:
                    if  q.search(y.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName"))):
                        call_conname_n_id[y.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName"))]=y.attrib.get(("{www.microsoft.com/SqlServer/Dts}DTSID"))
                        
                     
    return call_conname_n_id 

def get_mercury_sqltaks(fname,cnam_id):  #get sql task list with mercury as connection  
    root = ET.parse(fname,parser).getroot()
    mercury_sql_tasks=[]
    for x in root.iter('{www.microsoft.com/SqlServer/Dts}Executable'):
         if x.attrib.get(("{www.microsoft.com/SqlServer/Dts}Description"))=="Execute SQL Task":
                for y in x.iter('{www.microsoft.com/SqlServer/Dts}ObjectData'):
                    for sql in y.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}SqlTaskData')  :
                       # print (sql.attrib)
                        if (sql.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}Connection'))) in list(cnam_id.values()):
                            mercury_sql_tasks.append(x.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName")))
                            
                    
    return mercury_sql_tasks 

def update_pkg(fname,cnam_id,call_cnam_id,path,fname_only): #update xml file 
    print(fname_only)
    root = ET.parse(fname,parser)
#     root = re.sub(r'&lt;','<',root)
#     root = re.sub(r'&gt;','>',root)
    #root.text=None
    for x in root.iter('{www.microsoft.com/SqlServer/Dts}Executable'):
        x.text=None
        if x.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName")) in audit_end_tsks: # modify audit end task
             for y in x.iter('{www.microsoft.com/SqlServer/Dts}ObjectData'):
                    for sql in y.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}SqlTaskData')  :                 

                        if (sql.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}Connection'))) in list(cnam_id.values()):
                            sql.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}Connection']=list(call_cnam_id.values())[0]
                            sql.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}SqlStatementSource']="exec Create_AuditEvent ?,240,0,NULL,?,'Package Completed'"
                            for sql2 in sql.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterBinding'):
                                                               
                                if (sql2.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}DtsVariableName')))=="System::PackageName":
                                    #print('s')
                                    sql2.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterName']="1"
                            for sql3 in sql.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterBinding'):
                                if (sql3.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}DtsVariableName'))) in delete_var_list:
                                    sql.remove(sql3)
        if x.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName")) in audit_start_tsks: #modify audit start task
                for y in x.iter('{www.microsoft.com/SqlServer/Dts}ObjectData'):
                    for sql in y.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}SqlTaskData')  :                 

                        if (sql.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}Connection'))) in list(cnam_id.values()):
                            sql.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}Connection']=list(call_cnam_id.values())[0]
                            sql.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}SqlStatementSource']="Exec Create_AuditEvent ?,150,0,NULL,?,'Package Started'"
                            for sql2 in sql.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterBinding'):
                                                               
                                if (sql2.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}DtsVariableName')))=="System::PackageName":
                                    #print('s')
                                    sql2.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterName']="1"
                            for sql3 in sql.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterBinding'):
                                if (sql3.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}DtsVariableName'))) in delete_var_list :
                                    sql.remove(sql3)

                                    
        if x.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName")) in audit_error_tsks: #modify audit error task
                #print('err')
                for y in x.iter('{www.microsoft.com/SqlServer/Dts}ObjectData'):
                    for sql in y.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}SqlTaskData')  :                 

                        if (sql.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}Connection'))) in list(cnam_id.values()):
                            sql.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}Connection']=list(call_cnam_id.values())[0]
                            sql.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}SqlStatementSource']="EXEC Create_AuditEvent ?,  180, ?, ?, ?, ?"
                            for sql2 in sql.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterBinding'):
                                                               
                                if (sql2.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}DtsVariableName'))) not in delete_var_list +['User::ExtractDate']:
                                    val=int(sql2.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterName'])
                                    sql2.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterName']=str(val-1)
                            for sql3 in sql.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterBinding'):
                                if (sql3.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}DtsVariableName'))) in delete_var_list :
                                    sql.remove(sql3)
                        
        if x.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName")) in audit_rcount_tsks: #modify record count task
                #print('err')
                for y in x.iter('{www.microsoft.com/SqlServer/Dts}ObjectData'):
                    for sql in y.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}SqlTaskData')  :                 

                        if (sql.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}Connection'))) in list(cnam_id.values()):
                            sql.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}Connection']=list(call_cnam_id.values())[0]
                            sql.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}SqlStatementSource']="EXEC Create_AuditEvent ?, 210, ?, NULL, ?,  'New Rows'"
                            for sql2 in sql.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterBinding'):
                                                               
                                if (sql2.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}DtsVariableName'))) not in delete_var_list_rcount +['User::ExtractDate']:
                                    val=int(sql2.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterName'])
                                    sql2.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterName']=str(val-1)
                            for sql3 in sql.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterBinding'):
                                if (sql3.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}DtsVariableName'))) in delete_var_list :
                                    sql.remove(sql3)                                
                            
                            #print(sql.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}Connection'])
#         if i.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName")) in audit_start_tsks:
#             print (i.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName")))
#             print(i[0])
    print('Conversion complete for ', fname_only)
    
    root.write(os.path.join (directory+'_output',fname_only), xml_declaration=True,encoding='utf-8', method="xml")
            
                        
proc_calls={"call CreateAuditEvent(?,?,150,0,NULL,?,'Package Started')":"Exec Create_AuditEvent ?,150,0,NULL,?,'Package Started'",
"call CreateAuditEvent(?,?,240,?,NULL,?,'Package Completed')":"exec Create_AuditEvent ?,240,0,NULL,?,'Package Completed'",
"CALL CreateAuditEvent(?, ?, 180, ?, ?, ?, ?)":"EXEC Create_AuditEvent ?,  180, ?, ?, ?, ?",
 "CALL CreateAuditEvent(?, ?, 210, ?, NULL, ?,  'New Rows')"  : "EXEC Create_AuditEvent ?, 210, ?, NULL, ?,  'New Rows'"}   
pkg_sql_tasks={}
pkg_df_tasks={}
start_audit = re.compile(r"\L<words>", words=['Create Package Start Audi Event','Create Package Start Audit Event','Create Package Started Audit Event'])
for fname1 in os.listdir(directory):
    start_audit_event=[]
    fname= os.path.join (directory,fname1)          
    CallidusPkg,pkg_name=Check_Delaercomp_Comm(fname)
    #print(CallidusPkg,pkg_name)
    cnam_id={}
    call_cnam_id={}
    if CallidusPkg==1:
        #print(fname)
        cnam_id=get_mer_connname_ids(fname)
        #print(cnam_id)
        call_cnam_id=get_Callidus_connname_ids(fname)
        #print(call_cnam_id)
        if len(cnam_id)>=1 and len(call_cnam_id)>=1:
            mercury_sql_tasks_list=get_mercury_sqltaks(fname,cnam_id)
            #print(mercury_sql_tasks_list)
            audit_start_tsks=['Create Package Start Audi Event','Create Package Start Audit Event','Create Package Started Audit Event']
            audit_end_tsks=['Create Package Complete Audit Event','Create Package Completed Audit','Create Package Completed Audit Event','Create Package Complete Audit Event 1',' Create Package Complete Audit Event']
            delete_var_list=["User::ETLJobID","User::Row_Count"]
            delete_var_list_rcount=["User::ETLJobID"]
            audit_error_tsks=['Create OnError Audit Event']
            audit_rcount_tsks=[ 'Create RowCount Audit Event','Create RowCount Audit Event For Items','Create RowCount Audit Event For SOCRate','Create RowCount Audit Event record']
            start_audit_event=['yes' for i in mercury_sql_tasks_list if i in audit_start_tsks ]
            
            if len(start_audit_event) >0 :
                update_pkg(fname,cnam_id, call_cnam_id,directory,fname1)
            
           
 

m_Callidus_Subscriber_Dim.dtsx
Conversion complete for  m_Callidus_Subscriber_Dim.dtsx
m_SCRebate_Trans_load.dtsx
Conversion complete for  m_SCRebate_Trans_load.dtsx
PortInTransLoad.dtsx
Conversion complete for  PortInTransLoad.dtsx
Qpay_Payment_Fee_Summary.dtsx
Conversion complete for  Qpay_Payment_Fee_Summary.dtsx


In [27]:
a=''
len(a)

0

In [19]:
import lxml.etree as ET
import regex as re
import os
directory='C:\\balaji\\sourcecode\\etl-MPCSEDW\\MPCS_ETL_SSIS\\SSIS\\'
#directory='C:\\balaji\\2020\\teradata_migration_analysis\\pkgs\\'
#directory='C:\\balaji\\2020\\teradata_migration_analysis\\prod_pkgs_server\\'
#directory='C:\\balaji\\2020\\teradata_migration_analysis\\caltest\\'
parser = ET.XMLParser(strip_cdata=False,remove_blank_text=False)
ET.register_namespace('DTS',"www.microsoft.com/SqlServer/Dts")
ET.register_namespace('SQLTask',"www.microsoft.com/sqlserver/dts/tasks/sqltask")

def Check_Delaercomp_Comm(fname):    
    p = re.compile(r"\L<words>", words=['Dealer_Rpt1099_OLEDB_ConnString','DealerComp', 'DEALERCOMP', 'CALLIDUS', 'Callidus','Subsidy','SUBSIDY','Reporting','REPORTING'])
    n = re.compile(r"\L<words>", words=['None'])
    root = ET.parse(fname,parser).getroot()
    callidus=0
    #print(root.tag)
    #print(root.attrib)
    for x in root.iter('{www.microsoft.com/SqlServer/Dts}ConnectionManagers'):
        for y in x.iter('{www.microsoft.com/SqlServer/Dts}ConnectionManager'):
                if y.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName")) is not None:
                    if  p.search(y.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName"))):
                        #print(fname.split('\\')[-1])
                        callidus=1
                        break
    return callidus,fname.split('\\')[-1]                        
def get_mer_connname_ids(fname):    
    q = re.compile(r"\L<words>", words=['Mercury','MERCURY'])
    root = ET.parse(fname,parser).getroot()
    conname_n_id={}
    for x in root.iter('{www.microsoft.com/SqlServer/Dts}ConnectionManagers'):
        for y in x.iter('{www.microsoft.com/SqlServer/Dts}ConnectionManager'):
                if y.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName")) is not None:
                    if  q.search(y.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName"))):
                        conname_n_id[y.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName"))]=y.attrib.get(("{www.microsoft.com/SqlServer/Dts}DTSID"))
                        
                     
    return conname_n_id 

def get_mercury_sqltaks(fname,cnam_id):    
    root = ET.parse(fname,parser).getroot()
    mercury_sql_tasks=[]
    for x in root.iter('{www.microsoft.com/SqlServer/Dts}Executable'):
         if x.attrib.get(("{www.microsoft.com/SqlServer/Dts}Description"))=="Execute SQL Task":
                for y in x.iter('{www.microsoft.com/SqlServer/Dts}ObjectData'):
                    for sql in y.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}SqlTaskData')  :
                       # print (sql.attrib)
                        if (sql.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}Connection'))) in list(cnam_id.values()):
                            mercury_sql_tasks.append(x.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName")))
                            
                    
    return mercury_sql_tasks 

def get_mercury_dftaks(fname):    
    root = ET.parse(fname,parser).getroot()
    mercury_df_tasks=[]
    q = re.compile(r"\L<words>", words=['Mercury','MERCURY'])
    for x in root.iter('{www.microsoft.com/SqlServer/Dts}Executable'):
         if x.attrib.get(("{www.microsoft.com/SqlServer/Dts}Description"))=="Data Flow Task":
                for y in x.iter('{www.microsoft.com/SqlServer/Dts}ObjectData'):
                    for k in y.findall('.//connections//connection'):
                         if  q.search(k.attrib.get('connectionManagerID')):
                                mercury_df_tasks.append(x.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName")))
                                
                        
                            
                    
    return mercury_df_tasks 
                        
    
pkg_sql_tasks={}
pkg_df_tasks={}
for fname1 in os.listdir(directory):
    fname= os.path.join (directory,fname1)          
    CallidusPkg,pkg_name=Check_Delaercomp_Comm(fname)
    #print(CallidusPkg,pkg_name)
    cnam_id={}
    if CallidusPkg==1:
        cnam_id=get_mer_connname_ids(fname)
        print(CallidusPkg,pkg_name)
        #print(cnam_id)
        if len(cnam_id)>=1:
            #print('a')
            mercury_sql_tasks_list=get_mercury_sqltaks(fname,cnam_id)
            mercury_df_tasks_list=get_mercury_dftaks(fname)
            pkg_sql_tasks[pkg_name]=mercury_sql_tasks_list
            pkg_df_tasks[pkg_name]=mercury_df_tasks_list
 
with open('C:\\balaji\\2020\\teradata_migration_analysis\\Pkgs_with_Mercury_in_sqltasks_20200706_1.txt','w') as f:
    for i,j in pkg_sql_tasks.items():
        f.write("%s,%s\n" %(i, j))
with open('C:\\balaji\\2020\\teradata_migration_analysis\\Pkgs_with_Mercury_in_dftasks_20200706_1.txt','w') as f:
    for i,j in pkg_df_tasks.items():
        f.write("%s,%s\n" %(i, j))

1 1099_Load Data.dtsx
1 1099_Load_Final.dtsx
1 1099_PreLoad_Data.dtsx
1 AALTransLoad.dtsx
1 AALTransTrueupLoad.dtsx
1 AAL_Trans_Load.dtsx
1 BanFirstActivation.dtsx
1 BogoTransLoad.dtsx
1 Bogo_trans_Load.dtsx
1 BOHOTransLoad.dtsx
1 CallidusCredit.dtsx
1 Callidus_Daily_Proxy_List_Load.dtsx
1 Callidus_Daily_User_List_Load.dtsx
1 Callidus_DQ_Files_Check.dtsx
1 Callidus_Expired_Passwords_Report_Load.dtsx
1 Callidus_LDAP_Audit.dtsx
1 Callidus_PortData_Load.dtsx
1 Callidus_Rebate_Load.dtsx
1 CombinedLimitCheck.dtsx
1 Compensation_Extract_Load.dtsx
1 CustomerFeeTransLoad.dtsx
1 Daily_Neustar_Promo_Report_Load.dtsx
1 DC2_Audit_AuditFileGenerate.dtsx
1 DC2_Audit_QPayCorrectionFile.dtsx
1 DC2_Audit_QPayReconReport.dtsx
1 DC2_CallidusExtract.dtsx
1 DC2_ImportFromMicra.dtsx
1 DC2_ImportFromMIDAS.dtsx
1 DC2_ImportFromMIDAS_Terminated.dtsx
1 DC2_ImportFromNetezza.dtsx
1 DC2_ImportFromOIMDataFile.dtsx
1 DC2_ImportFromOracle.dtsx
1 DC2_QPayReconcile.dtsx
1 DC2_QPayReconcile_LoadExceptionFile.dtsx
1 DC2

In [15]:
import xml.etree.ElementTree as ET
import regex as re
import os
#directory='C:\\balaji\\2020\\teradata_migration_analysis\\caltest\\'
#directory='C:\\balaji\\2020\\teradata_migration_analysis\\pkgs'
directory='C:\\balaji\\2020\\teradata_migration_analysis\\prod_pkgs_server\\'


def Check_Delaercomp_Comm(fname):    
    p = re.compile(r"\L<words>", words=['DealerComp', 'DEALERCOMP', 'CALLIDUS', 'Callidus','Subsidy','SUBSIDY','Rpt1099'])
    n = re.compile(r"\L<words>", words=['None'])
    root = ET.parse(fname).getroot()
    #print(root.tag)
    #print(root.attrib)
    for x in root.iter('{www.microsoft.com/SqlServer/Dts}ConnectionManagers'):
        for y in x.iter('{www.microsoft.com/SqlServer/Dts}ConnectionManager'):
            for z in y.iter('{www.microsoft.com/SqlServer/Dts}ObjectData'):
                for m in z.iter('{www.microsoft.com/SqlServer/Dts}ConnectionManager'):
                     if  p.search(str(m.attrib.get(("{www.microsoft.com/SqlServer/Dts}ConnectionString")))):
                        print(fname.split('\\')[-1])
                        break

         
for fname1 in os.listdir(directory):
    fname= os.path.join (directory,fname1)          
    Check_Delaercomp_Comm(fname)

1099_Load_Final.dtsx
1099_PreLoad_Data.dtsx
1099_PreLoad_Data.dtsx
AALTransLoad.dtsx
AALTransTrueupLoad.dtsx
AAL_Trans_Load.dtsx
BanFirstActivation.dtsx
BogoTransLoad.dtsx
Bogo_trans_Load.dtsx
BOHOTransLoad.dtsx
CallidusCredit.dtsx
Callidus_Daily_Proxy_List_Load.dtsx
Callidus_Daily_User_List_Load.dtsx
Callidus_DQ_Files_Check.dtsx
Callidus_Expired_Passwords_Report_Load.dtsx
Callidus_LDAP_Audit.dtsx
Callidus_LDAP_Audit.dtsx
Callidus_PortData_Load.dtsx
Callidus_Rebate_extract.dtsx
Callidus_Rebate_Load.dtsx
Callidus_Rebate_Load.dtsx
CombinedLimitCheck.dtsx
Compensation_Extract_Load.dtsx
Compensation_Extract_Load.dtsx
CustomerFeeTransLoad.dtsx
Daily_Neustar_Promo_Report_Load.dtsx
DC2_ActiveMABLEDealerCode.dtsx
DC2_Audit_AuditFileGenerate.dtsx
DC2_Audit_QPayCorrectionFile.dtsx
DC2_Audit_QPayReconReport.dtsx
DC2_Audit_QPayReconReport.dtsx
DC2_Audit_QPayReconReport.dtsx
DC2_Audit_QPayReconReport.dtsx
DC2_Audit_QPayReconReport.dtsx
DC2_Audit_QPayReconReport.dtsx
DC2_CallidusExtract.dtsx
DC2_Cal

In [24]:
import xml.etree.ElementTree as ET
import regex as re
import os
directory='C:\\balaji\\sourcecode\\etl-MPCSEDW\\MPCS_ETL_SSIS\\SSIS\\'
#directory='C:\\balaji\\2020\\teradata_migration_analysis\\pkgs\\test'

def Check_Delaercomp_Comm(fname):    
    p = re.compile(r"\L<words>", words=['DealerComp', 'DEALERCOMP', 'CALLIDUS', 'Callidus','Subsidy','SUBSIDY','Reporting','REPORTING'])
    n = re.compile(r"\L<words>", words=['None'])
    root = ET.parse(fname).getroot()
    callidus=0
    #print(root.tag)
    #print(root.attrib)
    for x in root.iter('{www.microsoft.com/SqlServer/Dts}ConnectionManagers'):
        for y in x.iter('{www.microsoft.com/SqlServer/Dts}ConnectionManager'):
                if y.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName")) is not None:
                    if  p.search(y.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName"))):
                        #print(fname.split('\\')[-1])
                        callidus=1
                        break
    return callidus,fname.split('\\')[-1]                        
def check_for_Select(fname)  :
    select=0
    p = re.compile(r"\L<words>", words=['DealerComp', 'DEALERCOMP', 'CALLIDUS', 'Callidus','Subsidy','SUBSIDY','Rpt1099'])
    with open(fname,'r') as f:
        lines=f.readlines()
        for i in lines:
            #print(i)
            if  p.search(i,re.IGNORECASE):
                select=1
                #print(i)
                #print(fname)
                break
           
    return select
pkg_with_select=[]
for fname1 in os.listdir(directory):
    fname= os.path.join (directory,fname1)          
    ifselect=check_for_Select(fname)
    if ifselect==1:
        print(fname.split('\\')[-1])
        pkg_with_select.append(fname.split('\\')[-1])
            
 
with open('C:\\balaji\\2020\\teradata_migration_analysis\\Pkgs_with_callidus_20200706.txt','w') as f:
    for i in pkg_with_select:
        f.write("%s\n" %(i)) 

1099_Load Data.dtsx
1099_Load_Final.dtsx
1099_Oracle_data_2017.dtsx
1099_PreLoad_Data.dtsx
AALTransLoad.dtsx
AALTransTrueupLoad.dtsx
AAL_Trans_Load.dtsx
BanFirstActivation.dtsx
BogoTransLoad.dtsx
Bogo_trans_Load.dtsx
BOHOTransLoad.dtsx
CallidusCredit.dtsx
Callidus_Daily_Proxy_List_Load.dtsx
Callidus_Daily_User_List_Load.dtsx
Callidus_DQ_Files_Check.dtsx
Callidus_Expired_Passwords_Report_Load.dtsx
Callidus_LDAP_Audit.dtsx
Callidus_PortData_Load.dtsx
Callidus_Rebate_Load.dtsx
CombinedLimitCheck.dtsx
Compensation_Extract_Load.dtsx
CustomerFeeTransLoad.dtsx
Daily_Neustar_Promo_Report_Load.dtsx
DC2_ActiveMABLEDealerCode.dtsx
DC2_Audit_AuditFileGenerate.dtsx
DC2_Audit_QPayCorrectionFile.dtsx
DC2_Audit_QPayReconReport.dtsx
DC2_CallidusExtract.dtsx
DC2_ImportFromMicra.dtsx
DC2_ImportFromMIDAS.dtsx
DC2_ImportFromMIDAS_Terminated.dtsx
DC2_ImportFromNetezza.dtsx
DC2_ImportFromOIMDataFile.dtsx
DC2_ImportFromOracle.dtsx
DC2_QPayReconcile.dtsx
DC2_QPayReconcile_LoadExceptionFile.dtsx
DC2_QPayReconci

In [25]:
a=[]

In [26]:
len(a)

0

In [ ]:
a.appen